In [3]:
import pandas as pd
import re
from eunjeon import Mecab
import collections

In [4]:
dirr = 'C:/Users/rsh15/Google Drive/crawler_data/estate_news/'
#'C:/Users/rsh15/Google Drive/crawler_data/estate_news/'
#'C:/Users/shic/Desktop/shinhan_simon/google_drive/crawler_data/estate_news/'

In [5]:
news_df = pd.read_csv(dirr+'RAA_부동산114뉴스기사_유승훈.csv')
news_df = news_df.dropna()

In [7]:
news_df['yearmonth'] = news_df['NEWS_WRT_YMD'].str.slice(stop=7).str.replace('.','',regex=False)
news_df

,NEWS_TITLE_NM,NEWS_WRT_YMD,NEWS_MEDIA_NM,NEWS_BODY_CNTT,yearmonth
0,"[주간부동산] 매매·전세, 연말연시 맞아 `한산`",2012.01.01,이데일리,"- 매매, 서울 0.05%↓, 신도시 0.02%↓, 수도권 0.02%↓- 전세, 서...",201201
1,"연말 거래 한산, 매매도 전세도 약세 지속",2012.01.01,조선비즈,서울 아파트 값이 2주 연속 하락세다. 2011년 마지막 주 시장은 비수기로 한산했...,201201
2,"2012년 아파트 공급 줄고, 오피스텔 늘고",2012.01.01,헤럴드POP,"새해에는 브랜드 아파트 공급은 작년보다 줄고, 대표적인 수익형부동산 상품인 오피스텔...",201201
3,수도권 집 한 채에 지방 아파트 몇 채 산다고?,2012.01.01,헤럴드POP,수도권 아파트 한 채 값에 지방 아파트 여러 채를 산다는 말도 옛말이 됐다. 지방 ...,201201
4,고급빌라 맞수 청담동 vs 한남동…“유엔빌리지(한남동) 살아요” ‘70억 펜트하우스...,2012.01.01,매경이코노미,한남동 유엔빌리지. / 청담동 마크힐스. TV 드라마 속 부잣집에서 전화를 받는 사...,201201
...,...,...,...,...,...
45907,[부동산캘린더] 3기 신도시 사전청약 1순위 접수…분양시장 후끈,2021.07.31,연합뉴스,(서울=연합뉴스) 김동규 기자 = 다음 주에는 3기 신도시 사전청약 1순위 접수를 ...,202107
45908,[분양캘린더]8월 첫째 주 1만4368가구 분양…견본주택 3곳 개관,2021.07.31,뉴시스,전국 20개 단지…일반분양 1만86가구[서울=뉴시스] 31일 부동산114에 따르면 ...,202107
45909,올 상반기 청약률 상위 단지는 초등학교 도보 통학권,2021.07.31,데일리안,"분양 시장 주 수요층 3040세대, 자녀 교육 우선안전한 통학 환경 선호해올해 상반...",202107
45910,집값에 치이고 전셋값에 또 치이고,2021.07.31,비즈니스워치,[집값 톡톡]전셋값→집값 상승 악순환서울·수도권·지방까지 전셋값 상승 확산집값은 '...,202107


In [22]:
dates = news_df['yearmonth'].unique().tolist()
dates[0:10]

['201201',
 '201202',
 '201203',
 '201204',
 '201205',
 '201206',
 '201207',
 '201208',
 '201209',
 '201210']

In [75]:
def news_article_preprocess(article_cnt):
    content_txt = re.sub(r'[\(\[].*?[\)\]]','',article_cnt)
    content_txt = re.sub('<.*?>','',content_txt)
    content_txt = re.sub('＜.*?＞','',content_txt)
    content_txt = re.sub('\s{2,}','',content_txt)
    content_txt = content_txt.replace('부동산114','')
    return content_txt

In [76]:
news_article_preprocess(news_df['NEWS_BODY_CNTT'][100])

'부산지역 주간 아파트 가격이 2년 11개월 만에 하락해 부산 부동산 시장이 본격 하락세에 접어든 것이 아니냐는 우려를 낳고 있다.부산지역 주간 아파트 평균 가격은 지난 6일 기준으로 0.01% 하락했다. 이는 지난해 10월 28일에 나타났던 일부 지역의 하락세가 부산 전체로 확산된 것으로 장기 하락의 신호탄으로 분석된다. △아파트 가격 하락의 확산과 원인=부산지역 아파트 하락세는 서부산권에서 먼저 시작됐다. 지난해 10월 28일 기준으로 주간 아파트 가격을 살펴보면 부산 북구와 부산진구가 각각 0.01%와 0.04% 하락했다. 부산지역 아파트 가격이 3년 만에 하락한 것은 이때부터다. 이후 하락세는 사상구·강서구·사하구로 확산됐다.3년간 급등세 이후 매수 실종 유럽발 재정위기도 악재 작용 경기 침체로 하향세 지속될 듯지난해 10월 28일 이후 지난 6일까지 북구는 0.16%, 강서구는 0.24%, 사상구는 0.40% 하락했다. 반면 부산진구와 사하구는 하락과 상승을 반복하면서 0.33%와 0.06% 상승했다. 이들 지역이 먼저 하락한 것은 우선 지난 3년간 상승폭이 가장 컸기 때문이다. 지난 2009년 2월 6일 대비 지난해 10월 28일 기준으로 북구는 52.98%, 사상구는 65.15%, 사하구는 55.38% 상승했었다. 이는 같은 기간 부산지역 전체 상승률의 배 가까운 것이다.여기에 주거 환경이 개발 호재를 뒷받침 못했기 때문이라는 분석이다. 사상구는 부산~김해 경전철, 사상~하단 도시철도 개발로 인해 복합 역세권에 대한 기대가 강하게 작용했다. 강서구는 신항개발, 을숙도대교, 거가대로 개통으로 접근성이 개선되면서 큰 폭으로 가격이 상승했다. 하지만 사상구의 경우 최근 10년간 신축된 아파트가 없을 정도로 주거 환경이 낙후돼 높게 형성된 가격에 이사할 인구가 유입되지 않으면서 가격이 하락하기 시작한 것으로 분석된다. 강서구는 신규 입주 아파트들의 분양가가 높아 전체 가격이 올랐으나 입주 이후 가격이 하락하면서 조정을 받고 있다. 북구는 화명 주공 재건축

In [74]:
news_df['NEWS_BODY_CNTT'].map(news_article_preprocess)[23122]

'조세소위원회의     백승렬 기자 = 기획재정위원회 조세소위원장인 새누리당 강석훈 의원이 22일 국회에서 열린 조세소위원회의를 주재하고 있다.민주당 \'임대소득 과세 2년 유예\' 정부안에 제동…세법개정안 통과 불투명11.3대책 등 잇단 규제에 시장 위축…공급과잉속 \'경착륙\' 우려도 서미숙 기자 = 최근 침체 조짐을 보이는 주택시장에 또다른 대형 악재가 터질 전망이다.    정부가 세법개정안을 통해 오는 2018년 말까지 2년간 시행을 유예하기로 했던 2천만원 이하 주택 전월세 임대소득 과세가 야당의 반대로 무산될 위기에 놓인 때문이다.     부동산 업계는 앞으로 아파트 입주물량이 급증하고 11·3 등 정부의 잇따른 부동산 대책으로 주택시장이 얼어붙고 있는 가운데 내년부터 임대소득 과세마저 시행될 경우 주택시장이 급격하게 위축될 것으로 우려하고 있다.     27일 정부에 따르면 내달 초 열리는 국회 조세소위원회에서 2017년 세법개정안이 논의되는 가운데 임대소득 과세유예안의 운명이 갈릴 전망이다.     기재부는 지난 7월말 세법개정안을 발표하면서 당초 내년부터 시행될 2천만원 이하의 주택 임대소득자에 대한 과세를 2018년까지 2년간 유예하기로 했다.     임대소득자에 세 부담이 커질 경우 집주인들이 월세를 높이는 방법으로 세금을 전가할 수 있고 임대소득자의 건강보험이 직장보험에서 지역보험으로 전환돼 건강보험료 부담이 크게 늘어난다는 점에서 건보료 개편 이후 시행하는 것이 바람직하다는 판단에서다.     그러나 국회 다수당인 민주당이 이에 제동을 걸고 내년부터 즉시 과세를 요구하고 나서면서 법안 통과가 어려운 상황에 놓였다.    민주당은 "임대소득자에 대해 애초 계획대로 세금을 걷어야 한다"며 올해로 끝나는 일몰 연장에 반대하고 있다.    당초 비과세 일몰이 2년 유예될 것으로 기대했던 주택시장은 초긴장 상태다.     부동산 전문가들은 임대소득 과세에 대한 취지는 인정하지만 당장 시행할 경우 주택시장의 하방 리스크가 가속화할 것으로 우려하고 

In [66]:
df_content = pd.concat(list_tmp).reset_index(drop=True)
df_content

,dates,content
0,201201,"- 매매, 서울 0.05%↓, 신도시 0.02%↓, 수도권 0.02%↓- 전세, 서..."
1,201202,투자분석 보고서분담금 9000만원까지 절감1만 가구가 넘는 대단지로 변모하는 서울...
2,201203,"동영상 뉴스【 앵커멘트 】 김포 한강과 파주 운정, 그리고 인천 송도·청라 지구...."
3,201204,국내 최대 포털 네이버가 오픈마켓 서비스 ‘샵N’의 문을 열고 업계로부터 뭇매를 ...
4,201205,2008년 금융위기 이후 가격이 크게 내린 아파트 중에서 소액 투자가 가능한 중소형...
...,...,...
110,202103,"잠실 엘스, 2억 내린 전세매물 등장두달새 25% 증가…서울·경기 비슷“다만 입주 ..."
111,202104,"- 대전 하늘채 스카이앤 2차, 5월중 분양일정 순항… 상담 가능한 ‘사전홍보관’ ..."
112,202105,동영상 뉴스서울 아파트값이 재건축 단지를 중심으로 다시 들썩이는 조짐이 나타나자 오...
113,202106,매년 6월 1일 기준 보유세 부과6월부터 매물잠김 시작되는 이유올해는 양도세 중과까...


In [67]:
tagger = Mecab()
tagger_list = []
for roww in df_content.iterrows():
    tagged = tagger.nouns(roww[1]['content'])
    tagged = [s for s in tagged if len(s) > 1]
    tagged_counter = collections.Counter(tagged)
    tagged_common  = tagged_counter.most_common(100)
    
    word_list = []
    count_list = []
    for word,count in tagged_common:
        word_list.append(word)
        count_list.append(count)
    
    tagger_list.append(pd.DataFrame({
        'date'  : roww[1]['dates'],
        'word'  : word_list,
        'count' : count_list
    }))

In [68]:
tagger_100[tagger_100['date']=='201204']

,date,word,count
0,201204,성폭행,1
1,201204,가담,1
2,201204,아이돌,1
3,201204,멤버,1
4,201204,형사,1
5,201204,처벌,1
6,201204,검토,1
7,201204,이하이,1
8,201204,박지민,1
9,201204,백아연,1


In [12]:
tagger_100 = pd.concat(tagger_list)
tagger_100.to_csv(dirr+'wordcount_df_100.csv',index=False,encoding='UTF-8-sig')

In [118]:
tagger = Mecab()
tagger_list = []
for roww in df_content.iterrows():
    tagged = tagger.nouns(roww[1]['content'])
    tagged = [s for s in tagged if len(s) > 1]
    tagged_counter = collections.Counter(tagged)
    tagged_common  = tagged_counter.most_common(50)
    
    word_list = []
    count_list = []
    for word,count in tagged_common:
        word_list.append(word)
        count_list.append(count)
    
    tagger_list.append(pd.DataFrame({
        'date'  : roww[1]['dates'],
        'word'  : word_list,
        'count' : count_list
    }))

In [119]:
tagger_50 = pd.concat(tagger_list)
tagger_50.to_csv(dirr+'wordcount_df_50.csv',index=False,encoding='UTF-8-sig')